In order to estimate how many people live near each dam in the country, we will use two data sources: the Statistical Grid and the Census Tracts produced by the the Brazilian Institute of Geopgraphy and Statistics (IBGE).

The statistical grid is the highest resolution datapoint available for measuring the spacial distribution of the population. It's divided in 300m x 300m (urban areas) or 2km x 2km (rural area) squares, each one with an estimate of the population living there.

<div>
    <img src=notebook-assets/grid.png width="600">
</div>

The census tract differ in shape and size, following the geopolitical divisions of Brazil. Howevert, they also offer plenty of information about of the population that live inside them.

<img src="notebook-assets/tracts.jpg"><br>

We want to make the bost of both worlds: that is, we want to use the statistical grid to measure not only population counts in a very small area, but also the demographics. To do, we will use a technique called [areal weigthed interpolation](https://dges.carleton.ca/CUOSGwiki/index.php/Areal_Interpolation_in_Python_Using_Tobler), which assumes that the characeristics of the population are homogeneous inside each cenus tract.

One important caveat: since there is no Census in the country since 2010, both datasets are reflecting a population pattern that is now 12 years old. They are still the most up to date dataset that is available until the next Census, which is set to start on 2022

#### Importing packages

In [1]:
import geopandas as gpd
import pandas as pd
import tobler

#### Reading census tract data

The census tract data was already parsed in the notebooks `1`, `2` and `3` in this repository. We will simply read them in.

In [2]:
tracts = gpd.read_file("../data/brazil/censo/combined/combined.gpkg")

#### Reading statistical grid data
The statistical grid, for all it's granularity, is too big and too heavy to handle at once. To make up for this, IBGE divided it into many smaller files divided as shown in the following image:

<img src='notebook-assets/articulacao.jpg'> 

 Instead of reading it all into memory, we will define a function that reads one of those at each time and call it as needed.

In [3]:
def read_grid(id_):
    '''
    Reads and combines the population grids
    specified in the ids list into a single
    GeoDataFrame
    ---
    Parameters:
    
    ids -> A numerical id for the grids that should be read
    '''
    
    gdfs = []
    
    gdf = gpd.read_file(f"zip://../data/brazil/grade/grade_id{id_}.zip")
        
    return gdf
        

In [6]:
grid = read_grid('04')

Now we can call upon tobler to derive detailed population metrics for each one of the squares. 

In [8]:
interp = tobler.area_weighted.area_interpolate(source_df=tracts, 
                                                        target_df=grid,
                                                        extensive_variables=['total_residents'])

/Users/menegat/opt/anaconda3/envs/dams/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
interp.total_residents.max()

1145.0354192382656